# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [39]:
#pip install transformers datasets evaluate accelerate


#clear CUDA cache
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


# Set the device as GPU, MPS, or CPU according to availability
import torch
if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
    
    
#device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [40]:
import sys
import pprint
sys.displayhook = pprint.pprint

In [41]:
from datasets import load_dataset, load_dataset_builder,get_dataset_split_names,get_dataset_config_names

dataset_name = "dair-ai/emotion"

dataset = load_dataset(dataset_name)



splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset(dataset_name, split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(1000))


ds_builder = load_dataset_builder(dataset_name)
print(ds_builder.info.description)
print(ds_builder.info.features)

get_dataset_split_names(dataset_name)

configs = get_dataset_config_names(dataset_name)

print(configs)

ds


{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
['split', 'unsplit']


{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 1000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 1000
 })}

In [42]:
ds['test'][1]

{'text': 'i feel so thrilled to have three such distinguished individuals such as yourselves here',
 'label': 1}

In [43]:


num_classes = dataset['train'].features['label'].num_classes
id2label = {id:dataset['train'].features['label'].int2str(id) for id in range(num_classes)}
label2id = {label:id for (id,label) in id2label.items()}
print(ds_builder.info.features)
print(id2label)
print(label2id)
print(len(id2label))

{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}
{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}
6


In [44]:
from transformers import AutoTokenizer

foundation_model_name = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(foundation_model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})




def preprocess_function(examples):
    return tokenizer(examples["text"],padding="max_length", truncation=True)

tokenized_data = dataset.map(preprocess_function, batched=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

[101, 2096, 9670, 1999, 1996, 2406, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [45]:
from transformers import AutoModelForSequenceClassification



model = AutoModelForSequenceClassification.from_pretrained(
    foundation_model_name, 
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
)


for param in model.base_model.parameters():
    param.requires_grad = False
    
    
print(model.classifier)

model = model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=6, bias=True)


In [46]:
print(model)



DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [47]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

""" import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()} """

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

    # Calculate precision, recall, and F1-score
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    

batch_size = 1

# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        per_device_eval_batch_size=batch_size,
        label_names=["labels"],
    ),
    #train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

#trainer.train()

In [48]:
# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/500 [00:00<?, ?it/s]

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.817934513092041,
 'eval_accuracy': 0.032,
 'eval_precision': 0.1443512145748988,
 'eval_recall': 0.032,
 'eval_f1': 0.007531017078270091,
 'eval_runtime': 42.6607,
 'eval_samples_per_second': 23.441,
 'eval_steps_per_second': 11.72}

In [49]:
import pandas as pd
import numpy as np

df = pd.DataFrame(tokenized_ds["test"])
df = df[["text", "label"]]

# Replace <br /> tags in the text with spaces
df["text"] = df["text"].str.replace("<br />", " ")

# Add the model predictions to the dataframe
predictions = trainer.predict(tokenized_ds["test"])
df["predicted_label"] = np.argmax(predictions[0], axis=1)

df.head(10)

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/500 [00:00<?, ?it/s]

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,text,label,predicted_label
0,i was feeling really troubled and down over what my dad said,0,5
1,i feel so thrilled to have three such distinguished individuals such as yourselves here,1,5
2,i feel is that the most likeable characters aren t important enough to the plot,1,5
3,i tune out the rest of the world and focus on the rhythm of the needles and the softness of the yarn and for that time i feel my most peaceful,1,5
4,i sit here writing this i feel unhappy inside,0,5
5,im feeling and if ive liked being pregnant,2,5
6,im very hurt and i feel unimportant,0,5
7,i used to be able to hang around talk with the cashier when i was putting away my money now i feel rushed and stressed if i take a second to fumble with the coins and put them in my purse,3,5
8,i don t have the feeling of divine vibrations,1,5
9,i vented my feelings towards the pathetic excuse of a communicat,0,5


In [50]:
# Show full cell output
pd.set_option("display.max_colwidth", None)

df[df["label"] != df["predicted_label"]].head(10)

,text,label,predicted_label
0,i was feeling really troubled and down over what my dad said,0,5
1,i feel so thrilled to have three such distinguished individuals such as yourselves here,1,5
2,i feel is that the most likeable characters aren t important enough to the plot,1,5
3,i tune out the rest of the world and focus on the rhythm of the needles and the softness of the yarn and for that time i feel my most peaceful,1,5
4,i sit here writing this i feel unhappy inside,0,5
5,im feeling and if ive liked being pregnant,2,5
6,im very hurt and i feel unimportant,0,5
7,i used to be able to hang around talk with the cashier when i was putting away my money now i feel rushed and stressed if i take a second to fumble with the coins and put them in my purse,3,5
8,i don t have the feeling of divine vibrations,1,5
9,i vented my feelings towards the pathetic excuse of a communicat,0,5


In [51]:
# Show full cell output
pd.set_option("display.max_colwidth", None)

df[df["label"] == df["predicted_label"]].head(10)

,text,label,predicted_label
51,i feel a strange gratitude for the hated israeli occupation of sinai that lasted from to for actually recognizing the importance of sinais history,5,5
78,im feeling sentimental or in need of reassurance,0,0
134,i go through my day feeling your movements and am amazed that something so miraculous is happening in my body its like a special secret only you and i have,5,5
179,i was so uncomfortable and feeling weird feelings but wasn t sure if they were contractions since i never really felt contractions with jared until they jacked me up with pitocin,5,5
184,i feel shame in a strange way,5,5
193,i feel like i should not be surprised at this development,5,5
217,i received the blanket i was absolutely amazed on how fluffy it is and extremely soft i really didnt think it was going to feel that amazing,5,5
218,i feel more amazed and more thankful for having e in our lives,5,5
271,i feel shocked that you d stoup to destinys child b,5,5
293,i got home and told peter how i was feeling he wasnt shocked at all by what i was telling him,5,5


In [52]:
#clear CUDA cache
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

34

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [53]:
import peft
from peft import LoraConfig, get_peft_model, AutoPeftModel,PeftConfig

lora_config = LoraConfig(
    r=5, 
    lora_alpha=1, # the alpha parameter for the lora loss.
    target_modules= ['classifier'], # the modules that are to be trained.
    #target_modules="all-linear",
    lora_dropout=0.05, # the dropout rate for the lora loss.
    bias="lora_only", # the bias for the lora loss.
)

In [54]:
import torch
from transformers import Conv1D

def get_specific_layer_names(model):
    # Create a list to store the layer names
    layer_names = []
    
    # Recursively visit all modules and submodules
    for name, module in model.named_modules():
        # Check if the module is an instance of the specified layers
        if isinstance(module, (torch.nn.Linear, torch.nn.Embedding, torch.nn.Conv2d, Conv1D)):
            # model name parsing 

            layer_names.append('.'.join(name.split('.')[4:]).split('.')[0])
    
    return layer_names

list(set(get_specific_layer_names(model)))

['', 'attention', 'ffn']

In [55]:

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print (pytorch_total_params)

peft_model = get_peft_model(model, lora_config)
peft_model = peft_model.to(device)

print(peft_model.print_trainable_parameters())

print(peft_model)



595206
trainable params: 3,876 || all params: 66,961,956 || trainable%: 0.005788361379407734
None
PeftModel(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in

In [56]:
#Create a directory to contain the Model
import os
working_dir = './'

output_directory = os.path.join(working_dir, "./data/sentiment_analysis")

In [57]:

#Creating the TrainingArgs

from transformers import TrainingArguments, Trainer, DataCollatorWithPadding



trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-2, 
        # Reduce the batch size if you don't have enough memory
        #auto_find_batch_size=True,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=100,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        do_eval=True,
        do_predict=True,
        metric_for_best_model="accuracy",
        label_names=["labels"],
        #load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


trainer.train()

  0%|          | 0/50000 [00:00<?, ?it/s]

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'loss': 1.6554, 'grad_norm': 0.9739303588867188, 'learning_rate': 0.0198, 'epoch': 1.0}


  0%|          | 0/500 [00:00<?, ?it/s]

/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 1.2333229780197144, 'eval_accuracy': 0.533, 'eval_precision': 0.46200214190909067, 'eval_recall': 0.533, 'eval_f1': 0.4768362017539027, 'eval_runtime': 42.6792, 'eval_samples_per_second': 23.431, 'eval_steps_per_second': 11.715, 'epoch': 100.0}


/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'train_runtime': 11695.9555, 'train_samples_per_second': 8.55, 'train_steps_per_second': 4.275, 'train_loss': 1.35382287109375, 'epoch': 100.0}


TrainOutput(global_step=50000, training_loss=1.35382287109375, metrics={'train_runtime': 11695.9555, 'train_samples_per_second': 8.55, 'train_steps_per_second': 4.275, 'total_flos': 1.32488736768e+16, 'train_loss': 1.35382287109375, 'epoch': 100.0})

In [62]:
#Save the model.
#peft_model_path = os.path.join(output_directory, f"lora_model")

trainer.model.save_pretrained("peft_model_emotion", model_type="bert")

trainer.evaluate()


/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


  0%|          | 0/500 [00:00<?, ?it/s]

{'eval_loss': 1.2333229780197144,
 'eval_accuracy': 0.533,
 'eval_precision': 0.46200214190909067,
 'eval_recall': 0.533,
 'eval_f1': 0.4768362017539027,
 'eval_runtime': 42.426,
 'eval_samples_per_second': 23.57,
 'eval_steps_per_second': 11.785,
 'epoch': 100.0}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [63]:
#Load the Model.

peft_config = PeftConfig.from_pretrained("peft_model_emotion")
loaded_model = AutoPeftModel.from_pretrained(
                                        "peft_model_emotion",
                                        label2id=label2id,
                                        id2label=id2label,
                                        is_trainable=False,
                                        config=peft_config)

loaded_model = loaded_model.to(device)


/home/level-3/anaconda3/envs/genAI/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
#from transformers import pipeline

#classifier = pipeline( "sentiment-analysis",model=loaded_model,tokenizer=tokenizer)
text = "im happy"
inputs = tokenizer(text, return_tensors="pt")

inputs = inputs.to(device)

In [65]:
import torch

with torch.no_grad():
    logits = model(**inputs).logits
    
    
predicted_class_id = logits.argmax().item()
loaded_model.config.id2label[predicted_class_id]    

'joy'